In [2]:
path = 'REC NOT CLOSED VFinale2.xlsm'
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc
import numpy as np

In [5]:
def preprocess_data(path = 'RESOURCE SUMMARY_20220912.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)

    # Remove IOS/... prefix from Assignment name
    prefix_removed = wb['Assignment Name'].str.extract(r'(^IOS/[a-zA-Z0-9_\./]+)-([a-zA-Z0-9_\' -]+)')[1]
    idxs = ~wb['Assignment Name'].str.startswith('IOS/').fillna(False)
    prefix_removed[idxs] = wb[idxs]['Assignment Name']
    wb['Assignment Name'] = prefix_removed


    return wb

In [6]:
wb = preprocess_data()

In [7]:
def time_utilisation(wb):
   # exclude nonworking records
   df = wb.loc[wb['Reporting categories']!='Nonworking']
   df = df.rename(columns={'Reporting categories':'time'})

   # merge Time Category and Phase information in one -- activity
   non_audit = df[df['Time Category'].isna()]['Assignment Name']
   audit = df[~df['Time Category'].isna()]['Time Category']
   df['activity'] = pd.concat((non_audit,audit))

   # extract year from date, calculate working hours
   df['Date'] = pd.to_datetime(df['Date'])
   df = df.assign(year_month=df['Date'].dt.strftime('%B, %Y'))
   df = df.assign(working_hours=df[['Project Hours', 'Admin Hours']].sum(axis=1))

   df = df.groupby(['Resource','year_month','time','activity']).sum().drop(columns=['Project Hours', 'Admin Hours',
      'Nonworking Hours', 'Holiday', 'Total (All Entries)',
      'Utilization (All Entries)'])

   df['monthly_hours'] = df.groupby(level = [0,1]).sum()
   df['time_category_hours'] = df.groupby(level = [0,1,2]).sum()[['working_hours']]

   df['time_category_percentage']  = (100*df['time_category_hours']/df['monthly_hours']).round(1)
   df['activity_percentage'] = (100*df['working_hours']/df['time_category_hours']).round(1)

   df = df.reset_index()

   df.replace(np.nan,None,inplace=True)

   df = df.astype({'time_category_percentage':'string','activity_percentage':'string'})
   df['time_category_percentage']+='%'
   df['activity_percentage']+='%'

   df['time_stat'] = df['time_category_hours'].astype('string') + ' hours (' + df["time_category_percentage"] + ')'
   df['activity_hours']  =  df['working_hours'].astype('string') + ' hours (' + df["activity_percentage"] + ')'

   colour_map = dict(zip(df['time'].sort_values().unique(),px.colors.carto.Safe))
   colour_map['(?)'] = 'lightgrey'

   # hovertemplate='label = %{label}<br>%{color:.data}<br>Hours worked = %{value:.2f}'
   hovertemplate = 'label = %{label}<br>Activity = %{customdata[0]}<br>Working hours = %{value}<extra></extra>'
   fig = px.treemap(df,path=[px.Constant('Month'),'year_month','Resource','time','time_stat','activity','activity_hours'],
   hover_data=['time','activity_hours'],width = 1300,height=800,
   values='working_hours',color = 'time',color_discrete_map=colour_map,maxdepth=5)

   fig.update_layout(title='Time Utilization per Month and Auditor',title_x=0.5)
   fig.update_traces(hovertemplate=hovertemplate)

   return df,fig

In [6]:
df, fig = time_utilisation(wb)

In [7]:
names = wb['Resource'].dropna().unique()

In [8]:
def options(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

In [9]:
import dash_bootstrap_components as dbc

In [10]:
def drpdwn(i):
    opts = options(names)
    
    return dcc.Dropdown(options = opts, value = names,id = i,multi = True,className='dropdown')

In [11]:
dbc.themes.BOOTSTRAP

'https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css'

In [15]:
hovertemplate = 'label = %{label}<br>Activity = %{customdata[0]}<br>Working hours = %{value}<extra></extra>'
colour_map = dict(zip(df['time'].sort_values().unique(),px.colors.carto.Safe))
colour_map['(?)'] = 'lightgrey'

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])

server = app.server

app.layout = html.Div(children = [html.Div(
			children= [html.H1('Time Utilization per Month and Auditor',className='heading'),
					html.Div(drpdwn('names'),className='dropdown'),
					dcc.Graph(figure = fig, id='graph', className='figure')
				]
			)
		]
	)

@app.callback(
	Output('graph','figure'),
	Input('names','value'))
def update_figure(name):

	to_plot = df[df['Resource'].isin(name)]

	fig = px.treemap(to_plot,path=[px.Constant('Month'),'year_month','Resource','time','time_stat','activity','activity_hours'],
	hover_data=['time','activity_hours'],values='working_hours',color = 'time',color_discrete_map=colour_map,maxdepth=5,
	width = 1300,height=800)

	# fig.update_layout(title='Time Utilization per Month and Auditor',title_x=0.5)
	fig.update_traces(hovertemplate=hovertemplate)

	return fig
	
app.run_server()

Dash app running on http://127.0.0.1:8050/


In [15]:
fig.write_html('per_staff/time_utilisation.html')

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

df = px.data.tips()

# We have a list for every day
# In your case will be gropuby('RuleName')
# here for every element d
# d[0] is the name(key) and d[1] is the dataframe
dfs = list(df.groupby("day"))

first_title = dfs[0][0]
traces = []
buttons = []
for i,d in enumerate(dfs):
    visible = [False] * len(dfs)
    visible[i] = True
    name = d[0]
    traces.append(
        px.treemap(d[1],
                   path=['day', 'time', 'sex'],
                   values='total_bill').update_traces(visible=True if i==0 else False).data[0]
    )
    buttons.append(dict(label=name,
                        method="update",
                        args=[{"visible":visible},
                              {"title":f"{name}"}]))

updatemenus = [{'active':0, "buttons":buttons}]

fig = go.Figure(data=traces,
                 layout=dict(updatemenus=updatemenus))
fig.update_layout(title=first_title, title_x=0.5)
fig.show()

In [ ]:
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="None",
                     method="update",
                     args=[{"visible": [True, False, True, False]},
                           {"title": "Yahoo",
                            "annotations": []}]),
                dict(label="High",
                     method="update",
                     args=[{"visible": [True, True, False, False]},
                           {"title": "Yahoo High",
                            "annotations": [0]}]),
                dict(label="Low",
                     method="update",
                     args=[{"visible": [False, False, True, True]},
                           {"title": "Yahoo Low",
                            "annotations": [1]}]),
                dict(label="Both",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Yahoo",
                            "annotations": [1] + [1]}]),
            ]),
        )
    ])

In [99]:
fig.write_html('time_utilisation_updated.html')

# A profile of hours spent per assignment, and for each phase, see who charged time (ex Nairobi planning = 60% Tuyet-Mai, 30% Refiloe and 10% Carles etc)


In [8]:
def hours_per_assignment_year_phase(wb):
    df = wb[~wb['Phase'].isna()]

    df = df.assign(Working_hours = df[['Project Hours', 'Admin Hours']].sum(axis = 1))
    df.rename(columns = {'Working_hours':'Working Hours'},inplace=True)

    df['month'] = df['Date'].dt.strftime('%B')
    df['year'] = df['Date'].dt.strftime('%Y')

    # group by assignment, phase and resource
    work_share = (df.groupby(['year','Assignment Name','Phase','Resource'])
            .agg(working_hours = pd.NamedAgg(column="Total (All Entries)", aggfunc="sum")))

    work_share['assignment duration (hours)'] = work_share.groupby(level=[0,1]).sum()
    work_share['phase duration (hours)']=work_share.groupby(level=[0,1,2]).sum()[['working_hours']]

    work_share['phase_share'] = (100* work_share['working_hours']/work_share['phase duration (hours)'])
    work_share['display_phase_share'] = work_share['phase_share'].round(1)
    
    work_share = work_share.reset_index()

    colour_map = dict(zip(work_share['Phase'].sort_values().unique(),px.colors.sequential.Blugrn))
    colour_map['(?)'] = 'lightgrey'
    
    work_share.replace(np.nan,None,inplace=True)

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['display_phase_share'].astype('string') + '%'
    work_share['(Average) Working hours']=work_share['working_hours']

    phase_count = work_share.groupby(['year','Assignment Name','Phase']).count().reset_index(level=2)[['Phase']]
    work_share['Phase count'] = work_share['phase_share']/100

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['working_hours'].astype('string') + ' hours '\
    + '(' + work_share['display_phase_share'].astype('string') + '%)'


    hovertemplate='label = %{label}<br>Working hours = %{value}<br>Auditor = %{customdata[1]}<br>Phase share = %{customdata[0]}%<extra></extra>'
    fig = px.treemap(work_share,path = [px.Constant('Assignment'),'Assignment Name','year','Phase','staff_share'],
    color='Phase',branchvalues='total',maxdepth=4,hover_data=['display_phase_share','Resource'],
    values= 'working_hours',color_discrete_sequence=px.colors.sequential.Blugrn,color_discrete_map=colour_map,
    title='Profile of Hours Charged per Assignment, broken down by year then phase')

    fig.update_traces(root_color = 'lightgray')
    fig.update_layout(title_x= .5)
    fig.update_traces(hovertemplate=hovertemplate)

    return fig
    

In [30]:
fig = hours_per_assignment_year_phase(wb)

fig.write_html('hours_per_assignment_then_year.html')

In [31]:
fig

In [32]:
def hours_per_year(wb):
    df = wb[~wb['Phase'].isna()]

    df = df.assign(Working_hours = df[['Project Hours', 'Admin Hours']].sum(axis = 1))
    df.rename(columns = {'Working_hours':'Working Hours'},inplace=True)

    df['month'] = df['Date'].dt.strftime('%B')
    df['year'] = df['Date'].dt.strftime('%Y')

    # group by assignment, phase and resource
    work_share = (df.groupby(['year','Assignment Name','Phase','Resource'])
            .agg(working_hours = pd.NamedAgg(column="Total (All Entries)", aggfunc="sum")))

    work_share['assignment duration (hours)'] = work_share.groupby(level=[0,1]).sum()
    work_share['phase duration (hours)']=work_share.groupby(level=[0,1,2]).sum()[['working_hours']]

    work_share['phase_share'] = (100* work_share['working_hours']/work_share['phase duration (hours)'])
    work_share['display_phase_share'] = work_share['phase_share'].round(1)
    
    work_share = work_share.reset_index()

    colour_map = dict(zip(work_share['Phase'].sort_values().unique(),px.colors.sequential.Blugrn))
    colour_map['(?)'] = 'lightgrey'
    
    work_share.replace(np.nan,None,inplace=True)

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['display_phase_share'].astype('string') + '%'
    work_share['(Average) Working hours']=work_share['working_hours']

    phase_count = work_share.groupby(['year','Assignment Name','Phase']).count().reset_index(level=2)[['Phase']]
    work_share['Phase count'] = work_share['phase_share']/100

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['working_hours'].astype('string') + ' hours '\
    + '(' + work_share['display_phase_share'].astype('string') + '%)'


    hovertemplate='label = %{label}<br>Working hours = %{value}<br>Auditor = %{customdata[1]}<br>Phase share = %{customdata[0]}%<extra></extra>'
    fig = px.treemap(work_share,path = [px.Constant('Year'),'year','Assignment Name','Phase','staff_share'],
    color='Phase',branchvalues='total',maxdepth=4,hover_data=['display_phase_share','Resource'],
    values= 'working_hours',color_discrete_sequence=px.colors.sequential.Blugrn,color_discrete_map=colour_map,
    title='Profile of Hours Charged per Year')

    fig.update_traces(root_color = 'lightgray')
    fig.update_layout(title_x= .5)
    fig.update_traces(hovertemplate=hovertemplate)

    return fig
    

In [34]:
fig = hours_per_year(wb)

fig.write_html('hours_per_year.html')
fig

In [13]:
def hours_per_assignment_phase_year(wb):
    df = wb[~wb['Phase'].isna()]

    df = df.assign(Working_hours = df[['Project Hours', 'Admin Hours']].sum(axis = 1))
    df.rename(columns = {'Working_hours':'Working Hours'},inplace=True)

    df['month'] = df['Date'].dt.strftime('%B')
    df['year'] = df['Date'].dt.strftime('%Y')

    # group by assignment, phase and resource
    work_share = (df.groupby(['year','Assignment Name','Phase','Resource'])
            .agg(working_hours = pd.NamedAgg(column="Total (All Entries)", aggfunc="sum")))

    work_share['assignment duration (hours)'] = work_share.groupby(level=[0,1]).sum()
    work_share['phase duration (hours)']=work_share.groupby(level=[0,1,2]).sum()[['working_hours']]

    work_share['phase_share'] = (100* work_share['working_hours']/work_share['phase duration (hours)'])
    work_share['display_phase_share'] = work_share['phase_share'].round(1)
    
    work_share = work_share.reset_index()

    colour_map = dict(zip(work_share['Phase'].sort_values().unique(),px.colors.sequential.Blugrn))
    colour_map['(?)'] = 'lightgrey'
    
    work_share.replace(np.nan,None,inplace=True)

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['display_phase_share'].astype('string') + '%'
    work_share['(Average) Working hours']=work_share['working_hours']

    phase_count = work_share.groupby(['year','Assignment Name','Phase']).count().reset_index(level=2)[['Phase']]
    work_share['Phase count'] = work_share['phase_share']/100

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['working_hours'].astype('string') + ' hours '\
    + '(' + work_share['display_phase_share'].astype('string') + '%)'


    hovertemplate='label = %{label}<br>Working hours = %{value}<br>Auditor = %{customdata[1]}<br>Phase share = %{customdata[0]}%<extra></extra>'
    fig = px.treemap(work_share,path = [px.Constant('Assignment'),'Assignment Name','Phase','year','staff_share'],
    color='Phase',branchvalues='total',maxdepth=4,hover_data=['display_phase_share','Resource'],
    values= 'working_hours',color_discrete_sequence=px.colors.sequential.Blugrn,color_discrete_map=colour_map,
    title='Profile of Hours Charged per Assignment, broken down by phase then year')

    fig.update_traces(root_color = 'lightgray')
    fig.update_layout(title_x= .5)
    fig.update_traces(hovertemplate=hovertemplate)

    return fig
    

In [14]:
fig = hours_per_assignment_phase_year(wb)

In [15]:
fig

In [28]:
fig.write_html('per_assignment/assignment_profile_phase_then_year.html')

In [16]:
def hours_per_assignment_phase_year(wb):
    df = wb[~wb['Phase'].isna()]

    df = df.assign(Working_hours = df[['Project Hours', 'Admin Hours']].sum(axis = 1))
    df.rename(columns = {'Working_hours':'Working Hours'},inplace=True)

    df['month'] = df['Date'].dt.strftime('%B')
    df['year'] = df['Date'].dt.strftime('%Y')

    # group by assignment, phase and resource
    work_share = (df.groupby(['year','Assignment Name','Phase','Resource'])
            .agg(working_hours = pd.NamedAgg(column="Total (All Entries)", aggfunc="sum")))

    work_share['assignment duration (hours)'] = work_share.groupby(level=[0,1]).sum()
    work_share['phase duration (hours)']=work_share.groupby(level=[0,1,2]).sum()[['working_hours']]

    work_share['phase_share'] = (100* work_share['working_hours']/work_share['phase duration (hours)'])
    work_share['display_phase_share'] = work_share['phase_share'].round(1)
    
    work_share = work_share.reset_index()

    colour_map = dict(zip(work_share['Phase'].sort_values().unique(),px.colors.sequential.Blugrn))
    colour_map['(?)'] = 'lightgrey'
    
    work_share.replace(np.nan,None,inplace=True)

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['display_phase_share'].astype('string') + '%'
    work_share['(Average) Working hours']=work_share['working_hours']

    phase_count = work_share.groupby(['year','Assignment Name','Phase']).count().reset_index(level=2)[['Phase']]
    work_share['Phase count'] = work_share['phase_share']/100

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['working_hours'].astype('string') + ' hours '\
    + '(' + work_share['display_phase_share'].astype('string') + '%)'


    hovertemplate='label = %{label}<br>Working hours = %{value}<br>Auditor = %{customdata[1]}<br>Phase share = %{customdata[0]}%<extra></extra>'
    fig = px.treemap(work_share,path = [px.Constant('Assignment'),'Assignment Name','Phase','year','staff_share'],
    color='Phase',branchvalues='total',maxdepth=4,hover_data=['display_phase_share','Resource'],
    values= 'working_hours',color_discrete_sequence=px.colors.sequential.Blugrn,color_discrete_map=colour_map,
    title='Profile of Hours Charged per Assignment, broken down by phase then year')

    fig.update_traces(root_color = 'lightgray')
    fig.update_layout(title_x= .5)
    fig.update_traces(hovertemplate=hovertemplate)

    return fig
    

In [17]:
hours_per_assignment_phase_year(wb)

In [ ]:
def hours_per_assignment_phase_year(wb):
    df = wb[~wb['Phase'].isna()]

    df = df.assign(Working_hours = df[['Project Hours', 'Admin Hours']].sum(axis = 1))
    df.rename(columns = {'Working_hours':'Working Hours'},inplace=True)

    df['month'] = df['Date'].dt.strftime('%B')
    df['year'] = df['Date'].dt.strftime('%Y')

    # group by assignment, phase and resource
    work_share = (df.groupby(['year','Assignment Name','Phase','Resource'])
            .agg(working_hours = pd.NamedAgg(column="Total (All Entries)", aggfunc="sum")))

    work_share['assignment duration (hours)'] = work_share.groupby(level=[0,1]).sum()
    work_share['phase duration (hours)']=work_share.groupby(level=[0,1,2]).sum()[['working_hours']]

    work_share['phase_share'] = (100* work_share['working_hours']/work_share['phase duration (hours)'])
    work_share['display_phase_share'] = work_share['phase_share'].round(1)
    
    work_share = work_share.reset_index()

    colour_map = dict(zip(work_share['Phase'].sort_values().unique(),px.colors.sequential.Blugrn))
    colour_map['(?)'] = 'lightgrey'
    
    work_share.replace(np.nan,None,inplace=True)

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['display_phase_share'].astype('string') + '%'
    work_share['(Average) Working hours']=work_share['working_hours']

    phase_count = work_share.groupby(['year','Assignment Name','Phase']).count().reset_index(level=2)[['Phase']]
    work_share['Phase count'] = work_share['phase_share']/100

    work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['working_hours'].astype('string') + ' hours '\
    + '(' + work_share['display_phase_share'].astype('string') + '%)'


    hovertemplate='label = %{label}<br>Working hours = %{value}<br>Auditor = %{customdata[1]}<br>Phase share = %{customdata[0]}%<extra></extra>'
    fig = px.treemap(work_share,path = [px.Constant('Assignment'),'Assignment Name','Phase','year','staff_share'],
    color='Phase',branchvalues='total',maxdepth=4,hover_data=['display_phase_share','Resource'],
    values= 'working_hours',color_discrete_sequence=px.colors.sequential.Blugrn,color_discrete_map=colour_map,
    title='Profile of Hours Charged per Assignment, broken down by phase then year')

    fig.update_traces(root_color = 'lightgray')
    fig.update_layout(title_x= .5)
    fig.update_traces(hovertemplate=hovertemplate)

    return fig
    

In [35]:
hours_per_assignment_phase_year(wb).write_html('hours_per_assignment_then_phase_then_year.html')

In [19]:
df

,Team,Resource,Date,Assignment Code,Audit Plan Title,Assignment Name,Type,Phase,Time Category,Project Hours,Admin Hours,Nonworking Hours,Holiday,Total (All Entries),Utilization (All Entries),Comment,Reporting categories,Working Hours,month,year
0,IOS Audit Team,Alicia Alvarez-Gracia,2022-09-02,2022-FO-HAV,2022 Audit Plan,Audit of the UNESCO Regional Office in Havana,NaN,2-Fieldwork,NaN,7.5,0.0,0.0,0.0,7.5,0.002680,NaN,Audit Time,7.5,September,2022
1,IOS Audit Team,Alicia Alvarez-Gracia,2022-09-01,2022-FO-HAV,2022 Audit Plan,Audit of the UNESCO Regional Office in Havana,NaN,2-Fieldwork,NaN,7.5,0.0,0.0,0.0,7.5,0.002680,NaN,Audit Time,7.5,September,2022
2,IOS Audit Team,Alicia Alvarez-Gracia,2022-08-31,2022-FO-HAV,2022 Audit Plan,Audit of the UNESCO Regional Office in Havana,NaN,2-Fieldwork,NaN,7.5,0.0,0.0,0.0,7.5,0.002680,NaN,Audit Time,7.5,August,2022
3,IOS Audit Team,Alicia Alvarez-Gracia,2022-08-30,2022-FO-HAV,2022 Audit Plan,Audit of the UNESCO Regional Office in Havana,NaN,2-Fieldwork,NaN,7.5,0.0,0.0,0.0,7.5,0.002680,NaN,Audit Time,7.5,August,2022
4,IOS Audit Team,Alicia Alvarez-Gracia,2022-08-29,2022-FO-HAV,2022 Audit Plan,Audit of the UNESCO Regional Office in Havana,NaN,2-Fieldwork,NaN,7.5,0.0,0.0,0.0,7.5,0.002680,NaN,Audit Time,7.5,August,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448,IOS Audit Team,Tuyet-Mai Grabiel,2021-03-19,2020-FO-KAT,2021 Audit Plan,Audit of UNESCO's Office in Kathmandu,Field Audit,2-Fieldwork,NaN,2.0,0.0,0.0,0.0,2.0,0.000690,NaN,Audit Time,2.0,March,2021
10451,IOS Audit Team,Tuyet-Mai Grabiel,2021-03-18,2020-FO-KAT,2021 Audit Plan,Audit of UNESCO's Office in Kathmandu,Field Audit,3-Reporting,NaN,4.0,0.0,0.0,0.0,4.0,0.001380,NaN,Audit Time,4.0,March,2021
10454,IOS Audit Team,Tuyet-Mai Grabiel,2021-03-17,2020-FO-KAT,2021 Audit Plan,Audit of UNESCO's Office in Kathmandu,Field Audit,3-Reporting,NaN,2.5,0.0,0.0,0.0,2.5,0.000862,NaN,Audit Time,2.5,March,2021
10458,IOS Audit Team,Tuyet-Mai Grabiel,2021-03-16,2020-FO-KAT,2021 Audit Plan,Audit of UNESCO's Office in Kathmandu,Field Audit,3-Reporting,NaN,3.0,0.0,0.0,0.0,3.0,0.001035,NaN,Audit Time,3.0,March,2021


In [29]:
df = wb[~wb['Phase'].isna()]

df = df.assign(Working_hours = df[['Project Hours', 'Admin Hours']].sum(axis = 1))
df.rename(columns = {'Working_hours':'Working Hours'},inplace=True)

df['month'] = df['Date'].dt.strftime('%B')
df['year'] = df['Date'].dt.strftime('%Y')

# group by assignment, phase and resource
work_share = (df.groupby(['year','Assignment Name','Phase','Resource'])
        .agg(working_hours = pd.NamedAgg(column="Total (All Entries)", aggfunc="sum")))

work_share['assignment duration (hours)'] = work_share.groupby(level=[0,1]).sum()
work_share['phase duration (hours)']=work_share.groupby(level=[0,1,2]).sum()[['working_hours']]

work_share['phase_share'] = (100* work_share['working_hours']/work_share['phase duration (hours)'])
work_share['display_phase_share'] = work_share['phase_share'].round(1)

work_share = work_share.reset_index()

colour_map = dict(zip(work_share['Phase'].sort_values().unique(),px.colors.sequential.Blugrn))
colour_map['(?)'] = 'lightgrey'

work_share.replace(np.nan,None,inplace=True)

work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['display_phase_share'].astype('string') + '%'
work_share['(Average) Working hours']=work_share['working_hours']

phase_count = work_share.groupby(['year','Assignment Name','Phase']).count().reset_index(level=2)[['Phase']]
work_share['Phase count'] = work_share['phase_share']/100

work_share['staff_share'] = work_share['Resource'] + ', ' + work_share['working_hours'].astype('string') + ' hours '\
+ '(' + work_share['display_phase_share'].astype('string') + '%)'


hovertemplate='label = %{label}<br>Working hours = %{value}<br>Auditor = %{customdata[1]}<br>Phase share = %{customdata[0]}%<extra></extra>'
fig = px.treemap(work_share,path = [px.Constant('Assignment'),'Assignment Name','Phase', 'Resource','year','display_phase_share'],
color='Phase',branchvalues='total',maxdepth=4,hover_data=['display_phase_share','Resource'],
values= 'working_hours',color_discrete_sequence=px.colors.sequential.Blugrn,color_discrete_map=colour_map,
title='Profile of Hours Charged per Assignment, broken down by phase then Auditor')

fig.update_traces(root_color = 'lightgray')
fig.update_layout(title_x= .5)
fig.update_traces(hovertemplate=hovertemplate)



In [ ]:
import plotly.graph_objects as go
fig = go.Figure(go.Scatter(x=[0, 1], y=[10, 6], mode='lines+markers+text', 
                           text=['start', 'end'], textposition=['middle left', 'middle right']))
fig.add_shape(type='line', x0=0, x1=0, y0=0, y1=1, xref='x', yref='paper')
fig.add_shape(type='line', x0=1, x1=1, y0=0, y1=1, xref='x', yref='paper')
fig.show()